<a href="https://colab.research.google.com/github/raamiiChu/NCCU_110-2_Group26_Applications-of-mathematics-softwares/blob/main/%E6%B8%AC%E8%A9%A6%E7%94%A8/%E6%B8%AC%E8%A9%A6DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
from tensorflow.keras.utils import to_categorical  # one hot encoding
from tensorflow.keras.models import Sequential  # 函數學習機
from tensorflow.keras.layers import Dense, Embedding  # 隱藏層
from tensorflow.keras.optimizers import Adam  # 優化器(學習方法)

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM

# <font color=#FF3030>先把 pt 檔上傳後，再去執行之後的程式碼</font>

##### 資料

把資料從 torch.tensor 轉成 numpy.array

In [ ]:
x_train = torch.load("train_emb.pt")
x_train = np.array([data.numpy().reshape(256,) for data in x_train])

In [ ]:
x_valid = torch.load("valid_emb.pt")
x_valid = np.array([data.numpy().reshape(256,) for data in x_valid])

In [ ]:
x_test = torch.load("test_emb.pt")
x_test = np.array([data.numpy().reshape(256,) for data in x_test])

資料數量沒有調整過，因此比例相當懸殊

In [ ]:
y_train = np.array([0]*3507 + [1]*14227 + [2]*722)
y_train = to_categorical(y_train, 3)

In [ ]:
y_valid = np.array([0]*2895 + [1]*7517 + [2]*586)
y_valid = to_categorical(y_valid, 3)

In [ ]:
y_test = np.array([0]*2895 + [1]*7622 + [2]*632)
y_test = to_categorical(y_test, 3)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(18456, 256) (18456, 3)
(10998, 256) (10998, 3)
(11149, 256) (11149, 3)


##### 組裝模型

In [ ]:
x_train[0].shape

(256,)

參數是亂寫的

In [ ]:
model = Sequential()
model.add(Dense(64,input_shape=(256,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 幾個結果

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.005), 
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 58,563
Trainable params: 58,563
Non-trainable params: 0
_________________________________________________________________


##### 使用模型

準確率極高，但不排除是因為資料比例懸殊的緣故

In [ ]:
model.fit(x_train, y_train, batch_size=10, epochs=10,
          validation_data=(x_valid, y_valid))

Epoch 1/10
1846/1846 [==============================] - 10s 4ms/step - loss: 0.0826 - accuracy: 0.9820 - val_loss: 0.1227 - val_accuracy: 0.9723
Epoch 2/10
1846/1846 [==============================] - 6s 4ms/step - loss: 0.0840 - accuracy: 0.9850 - val_loss: 0.1759 - val_accuracy: 0.9551
Epoch 3/10
1846/1846 [==============================] - 7s 4ms/step - loss: 0.0598 - accuracy: 0.9861 - val_loss: 0.1241 - val_accuracy: 0.9663
Epoch 4/10
1846/1846 [==============================] - 6s 4ms/step - loss: 0.0517 - accuracy: 0.9873 - val_loss: 0.1664 - val_accuracy: 0.9694
Epoch 5/10
1846/1846 [==============================] - 7s 4ms/step - loss: 0.0520 - accuracy: 0.9884 - val_loss: 0.2473 - val_accuracy: 0.9694
Epoch 6/10
1846/1846 [==============================] - 6s 3ms/step - loss: 0.0410 - accuracy: 0.9890 - val_loss: 0.1735 - val_accuracy: 0.9740
Epoch 7/10
1846/1846 [==============================] - 7s 4ms/step - loss: 0.0338 - accuracy: 0.9893 - val_loss: 0.3289 - val_accuracy

In [ ]:
y_predict = np.argmax(model.predict(x_test), axis=-1)

In [ ]:
y_predict

array([0, 0, 0, ..., 1, 2, 2])

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=1)

11149/11149 [==============================] - 25s 2ms/step - loss: 0.2062 - accuracy: 0.9507


In [ ]:
n = 1000
print(np.argmax(y_test[n]), y_predict[n], sep="|")

0|0
